In [1]:
################################################ Load 파일  ##########################################################

In [2]:
## 
import tensorflow as tf
import numpy as np
import pandas as pd 
from sklearn.model_selection import train_test_split
tf.random.set_seed(777)
import matplotlib.pyplot as plt
from numpy import argmax
import os
from sklearn.metrics import average_precision_score
import datetime
from sklearn.preprocessing import MinMaxScaler
import math
from pandas import DataFrame
import random
import csv
from tensorflow import keras
from keras.layers.merge import concatenate

import tensorflow.keras.backend as K
from keras.layers import Concatenate

N_batch = 10000 # batch_size
N_epochs = 5 # Epoch length
N_avg = 5 # ex) N개마다 평균 -> 2000/5 => 400
Input_size = 405

Using TensorFlow backend.


In [3]:
############################################### 모델 불러오기 함수 ######################################################
def load_model():
    model.load_weights(f"checkpoint/cp-{1}.ckpt")
################################################### AUPRC 함수  ####################################################    
def Metric_Aprecision(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision  

def get_data(percent, Training_zero, Training_one, Test_data_X, num_zero, num_one):
    print("데이터 처리중...")
    #1.OO, 2.OC, 3.CO, 4.CC

    train_zero_data_origin = Training_zero
    train_one_data_origin = Training_one
    test_data_origin = Test_data_X
    
    data_percent = int(100 / percent) #100에 딱 떨어지는 몫이 되어야 가능함
    
    # get split length
    zero_val_len = int(len(train_zero_data_origin) / data_percent) 
    one_val_len = int(len(train_one_data_origin) / data_percent)
    # print(zero_val_len, one_val_len, len(train_zero_data_origin), len(train_one_data_origin))

    train_zero_data = np.array(train_zero_data_origin)
    train_one_data = np.array(train_one_data_origin)

    for i in range(data_percent):
        zero_val = []
        one_val = []
        for j in range(data_percent):
            zero_val.append(train_zero_data[zero_val_len*j:zero_val_len*(j+1), :])
            one_val.append(train_one_data[one_val_len*j:one_val_len*(j+1), :])
    
    ## validation data
    zero_val_data = zero_val[num_zero]
    one_val_data = one_val[num_one]
    
    ## train data set(except validation data)
    train_zero = zero_val
    del train_zero[num_zero]  # used valdation set delete from train_data_set

    train_one = one_val
    del train_one[num_one] # used valdation set delete from train_data_set
    
    #list to array
    train_zero_origin = np.array(train_zero)
    train_one_origin = np.array(train_one)
    
    #2d array -> 1d array
    train_zero = [element for array in train_zero_origin for element in array]
    train_one =[element for array in train_one_origin for element in array]
    
    ## train_set
    train_zero = np.array(train_zero)
    train_one = np.array(train_one)
    train_set = np.vstack((train_zero, train_one))

    ## validation set
    zero_val_data = np.array(zero_val_data)
    one_val_data = np.array(one_val_data)
    val_set = np.vstack((zero_val_data, one_val_data))
    
    def shuffle_data(zero_data, one_data):    
        # parameters
        N = len(zero_data)
        K = len(one_data)
        L = int(np.ceil(N / K))
        Q = N // L
        R = N % L
        U = K - Q

        total_seq = []
        num = 0
        for i in range(Q):
            if i > Q-U:
                zero_seq = zero_data[L*i:L*(i+1),:]
                one_seq = one_data[Q-U+(2*num):Q-U+(2*(num+1)) ,:]
                shuffle_seq = np.vstack((zero_seq, one_seq))          
                np.random.shuffle(shuffle_seq)
                total_seq.append(shuffle_seq) 
                num = num + 1
            else:
                zero_seq = zero_data[L*i:L*(i+1),:]
                one_seq = one_data[i, :]
                shuffle_seq = np.vstack((zero_seq, one_seq))          
                np.random.shuffle(shuffle_seq)
                total_seq.append(shuffle_seq)  
            if i == Q-1:
                total_seq.append((zero_data[L*(i+1):,:]))
        
#         print("shuffle 데이터 확인 : ", total_seq[Q-1])
        total_seq = np.array(total_seq)
        # 2d list to 1d list and convert 2d array likely (none, 404)
        total_seq = [element for array in total_seq for element in array]
        total_seq = np.array(total_seq)
        return total_seq
    
    #train_set
    train_set = shuffle_data(train_zero, train_one)
    x_data = train_set[:, :-1]
    y_data = train_set[:, -1]

    # validation_set(divide label)
    val_set = shuffle_data(zero_val_data, one_val_data)
    val_x_data = val_set[:, :-1]
    val_y_data = val_set[:, [-1]]
    
    input_shape = len(x_data[0])
    x_data = x_data.reshape(x_data.shape[0], input_shape, 1)
    val_x_data = val_x_data.reshape(val_x_data.shape[0], input_shape, 1)
    
    test_data = np.array(test_data_origin)
    test_data = test_data.reshape(test_data.shape[0], input_shape, 1)
    
#     print("x_data의 shape 확인 : ", x_data.shape)
#     print("val_x_data의 shape 확인 : ", val_x_data.shape)
#     print("test_data의 shape 확인 : ", test_data.shape)
    
    print("데이터 처리 완료")
    return x_data, y_data, val_x_data, val_y_data, test_data

In [4]:
Training_zero = pd.read_csv('zero_data.csv', header=None)
Training_one = pd.read_csv('one_data.csv', header=None)
Test_data_X = pd.read_csv('test_data.csv', header=None)

In [5]:
checkpoint_path = "Best_model/cp-{epoch:d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)
cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path, save_weights_only=True, verbose=1,period=1)

In [6]:
x_data, y_data, val_x_data, val_y_data, test_data = get_data(10, Training_zero, Training_one, Test_data_X, 0, 0)
Learning_rate = 0.0001

데이터 처리중...
데이터 처리 완료


In [7]:
################################################ CNN 1D ###########################################################
model = tf.keras.Sequential([
    tf.keras.layers.Conv1D(input_shape=(x_data.shape[1],1), kernel_size=10, filters=60, strides=2, padding='same', activation='relu'),
    tf.keras.layers.MaxPool1D(pool_size=1),
    tf.keras.layers.Conv1D(kernel_size=5, filters=50, strides=2, padding='same', activation='relu'),
    tf.keras.layers.MaxPool1D(pool_size=1),
    tf.keras.layers.Conv1D(kernel_size=5, filters=50, strides=2, padding='same', activation='relu'),
    tf.keras.layers.MaxPool1D(pool_size=1),
    tf.keras.layers.Conv1D(kernel_size=5, filters=50, strides=2, padding='same', activation='relu'),
    tf.keras.layers.MaxPool1D(pool_size=1),
    tf.keras.layers.Conv1D(kernel_size=5, filters=50, strides=2, padding='same', activation='relu'),
    tf.keras.layers.MaxPool1D(pool_size=1),
    tf.keras.layers.Conv1D(kernel_size=5, filters=50, strides=2, padding='same', activation='relu'),
    tf.keras.layers.MaxPool1D(pool_size=1),
    tf.keras.layers.Conv1D(kernel_size=5, filters=50, strides=2, padding='same', activation='relu'),
    tf.keras.layers.MaxPool1D(pool_size=1),
    tf.keras.layers.Conv1D(kernel_size=5, filters=50, strides=2, padding='same', activation='relu'),
    tf.keras.layers.MaxPool1D(pool_size=1),
    tf.keras.layers.Dense(units=8, activation='relu'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=1, activation='sigmoid')
])
model.compile(optimizer=tf.keras.optimizers.Adam(lr=Learning_rate), loss='binary_crossentropy',metrics=['accuracy',Metric_Aprecision])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 202, 60)           660       
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 202, 60)           0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 101, 50)           15050     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 101, 50)           0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 51, 50)            12550     
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 51, 50)            0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 26, 50)            1

In [8]:
model.load_weights(f"checkpoint/cp-{1}.ckpt")
N_batch = 10000
N_epochs = 1
out_put1 = model.fit(x_data, y_data, batch_size = N_batch, validation_data = (val_x_data, val_y_data), callbacks=[cp_callback], epochs=N_epochs)

Train on 514619 samples, validate on 57179 samples
510000/514619 [============================>.] - ETA: 0s - loss: 0.1767 - accuracy: 0.9350 - Metric_Aprecision: 0.6554
Epoch 00001: saving model to Best_model/cp-1.ckpt
514619/514619 [==============================] - 15s 28us/sample - loss: 0.1768 - accuracy: 0.9350 - Metric_Aprecision: 0.6556 - val_loss: 0.1820 - val_accuracy: 0.9358 - val_Metric_Aprecision: 0.6560


In [9]:
#train set AUPRC
pred = model.predict(x_data, batch_size = N_batch)
val_pred  = model.predict(val_x_data, batch_size = N_batch)
AUPRC = average_precision_score(y_data, pred)
AUPRC_val = average_precision_score(val_y_data, val_pred)
print('Train SET AUPRC score : ', AUPRC)
print('Val SET AUPRC score : ', AUPRC_val)

Train SET AUPRC score :  0.5225811318824329
Val SET AUPRC score :  0.5222554273613098


In [10]:
# test set AUPRC
test_output = model.predict(test_data, batch_size = N_batch)
np.savetxt('Validation_output.csv', test_output)

In [11]:
model.load_weights(f"checkpoint/cp-{1}.ckpt")

In [12]:
pred = model.predict(x_data, batch_size = N_batch)
val_pred  = model.predict(val_x_data, batch_size = N_batch)
AUPRC = average_precision_score(y_data, pred)
AUPRC_val = average_precision_score(val_y_data, val_pred)
print('Train SET AUPRC score : ', AUPRC)
print('Val SET AUPRC score : ', AUPRC_val)

Train SET AUPRC score :  0.519791637100393
Val SET AUPRC score :  0.5240595537800274


In [13]:
test_output = model.predict(test_data, batch_size = N_batch)
np.savetxt('Final_Report_output.csv', test_output)